<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el libro "El Quijote de la mancha" de Miguel de Cervantes Saavedra

In [2]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt', sep='/n', engine='python', header=None)
df[100:105]

,0
100,Capítulo 10: De los graciosos razonamientos q...
101,Ya en este tiempo se había levantado Sancho P...
102,Seguíale Sancho a todo trote de su jumento; p...
103,"Todo esto fuera bien escusado, respondió Don ..."
104,menester yo más para pasar esta vida honrada ...


In [3]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2185


### 1 - Preprocesamiento

In [4]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [5]:
# Demos un vistazo
sentence_tokens[50:52]

[['¿dónde',
  'estáis',
  'señora',
  'mía',
  'que',
  'no',
  'te',
  'duele',
  'mi',
  'mal'],
 ['o', 'no', 'lo', 'sabes', 'señora']]

### 2 - Crear los vectores (word2vec)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [7]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                    window=2,       # cant de palabras antes y desp de la predicha
                    vector_size=400,       # dimensionalidad de los vectores 
                    negative=20,    # cantidad de negative samples... 0 es no se usa
                    workers=1,      # si tienen más cores pueden cambiar este valor
                    sg=1)           # modelo 0:CBOW  1:skipgram

In [8]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [9]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2185


In [10]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.key_to_index))

Cantidad de words distintas en el corpus: 3140


### 3 - Entrenar el modelo generador

In [12]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=80,
                compute_loss = True,
                callbacks=[callback()]
                )

Loss after epoch 0: 1083244.625
Loss after epoch 1: 996433.125
Loss after epoch 2: 911463.25
Loss after epoch 3: 907823.75
Loss after epoch 4: 855023.75
Loss after epoch 5: 829288.0
Loss after epoch 6: 823931.5
Loss after epoch 7: 819626.0
Loss after epoch 8: 815392.5
Loss after epoch 9: 786275.5
Loss after epoch 10: 756097.0
Loss after epoch 11: 754344.0
Loss after epoch 12: 751106.0
Loss after epoch 13: 746617.0
Loss after epoch 14: 744757.0
Loss after epoch 15: 739868.0
Loss after epoch 16: 738244.0
Loss after epoch 17: 734909.0
Loss after epoch 18: 731367.0
Loss after epoch 19: 731828.0
Loss after epoch 20: 712991.0
Loss after epoch 21: 675242.0
Loss after epoch 22: 671948.0
Loss after epoch 23: 665314.0
Loss after epoch 24: 668394.0
Loss after epoch 25: 664784.0
Loss after epoch 26: 665902.0
Loss after epoch 27: 663008.0
Loss after epoch 28: 661262.0
Loss after epoch 29: 656412.0
Loss after epoch 30: 660056.0
Loss after epoch 31: 659252.0
Loss after epoch 32: 655598.0
Loss after e

(8946360, 14964480)

### 4 - Ensayar

In [13]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["sancho"], topn=10)

[('panza', 0.4518609642982483),
 ('¡válame', 0.35024863481521606),
 ('sacando', 0.35015472769737244),
 ('–pues', 0.33013731241226196),
 ('majadero', 0.3240571618080139),
 ('calla', 0.3113749921321869),
 ('librillo', 0.31105220317840576),
 ('coloquio', 0.3030961751937866),
 ('maese', 0.3000044822692871),
 ('lloraba', 0.2983715534210205)]

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dulcinea"], topn=10)

[('toboso', 0.3881809711456299),
 ('argamasilla', 0.3628186881542206),
 ('desdeñado', 0.3595760464668274),
 ('micomicona', 0.35776427388191223),
 ('doña', 0.3542179763317108),
 ('académico', 0.3533497452735901),
 ('enemiga', 0.34580737352371216),
 ('amiga', 0.3452490270137787),
 ('señora', 0.3365603983402252),
 ('maría', 0.32746922969818115)]

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["caballero"], topn=10)

[('andante', 0.37290701270103455),
 ('hircania', 0.3609199523925781),
 ('príncipe', 0.34680935740470886),
 ('rodrigo', 0.33090540766716003),
 ('valeroso', 0.32336291670799255),
 ('español', 0.3160706162452698),
 ('–así', 0.31249138712882996),
 ('alcornoque', 0.3058152496814728),
 ('hidalgo', 0.30298566818237305),
 ('venturoso', 0.30040666460990906)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["molinos"], topn=10)

[('cuarenta', 0.4472845792770386),
 ('pobres', 0.4423711895942688),
 ('viento', 0.43055960536003113),
 ('hojas', 0.4302878677845001),
 ('dedos', 0.42937320470809937),
 ('modos', 0.4180419445037842),
 ('buscase', 0.4138668477535248),
 ('batanes', 0.4033724367618561),
 ('bienes', 0.39335986971855164),
 ('enmienda', 0.3857909142971039)]

### 5 - Visualizar agrupación de vectores

In [18]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [24]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=250
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer='colab') # esto para plotly en colab

### Conclusiones

- Se observaron algunos resultados interesantes al buscar palabras relacionadas:
* quijote -> don, dijo, respondió
* vuestra -> merced 
* Dios -> verdad
* rocinante -> caballo
- Se realizaron pruebas con diferentes cantidad de palabras graficada. Hay un gran concentración de palabras, esto puede significar que hay un gran conjunto de palabras relacionadas.